In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from alarmnetclass import AlarmNet  # Assuming this is the AlarmNet class from earlier
from federatedLearn import FederatedLearning

path = 'smoke_detection_iot.csv'

data = pd.read_csv(path).drop(['Unnamed: 0', 'UTC', 'CNT'], axis=1)
Y_df = data['Fire Alarm']
X_df = data.drop('Fire Alarm', axis=1)

Y_test_raw = X_df.values
Y_raw = Y_df.values

X_train_raw, X_test_raw, Y_train_raw, Y_test_raw = train_test_split(Y_test_raw, Y_raw, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_raw = scaler.fit_transform(X_train_raw)
X_test_raw = scaler.transform(X_test_raw)

X_train = torch.tensor(X_train_raw, dtype=torch.float32)
X_test = torch.tensor(X_test_raw, dtype=torch.float32)
Y_train = torch.tensor(Y_train_raw, dtype=torch.float32).reshape(-1, 1)
Y_test = torch.tensor(Y_test_raw, dtype=torch.float32).reshape(-1, 1)

In [ ]:
global_model = AlarmNet(num_features=X_train.shape[1], hidden_layers=[32, 16, 8])
federated_system = FederatedLearning(global_model, n_clients=3)

federated_system.split_data(X_train, Y_train)

federated_system.federated_training(epochs_per_round=100, lr=0.001, n_rounds=3)

federated_system.evaluate_global_model(X_test, Y_test)

--- Federated Training Round 1 ---
Client 1 - Epoch 25/100, Loss: 0.6769
Client 1 - Epoch 50/100, Loss: 0.6355
Client 1 - Epoch 75/100, Loss: 0.5681
Client 1 - Epoch 100/100, Loss: 0.4577
Client 2 - Epoch 25/100, Loss: 0.6765
Client 2 - Epoch 50/100, Loss: 0.6349
Client 2 - Epoch 75/100, Loss: 0.5677
Client 2 - Epoch 100/100, Loss: 0.4581
Client 3 - Epoch 25/100, Loss: 0.6761
Client 3 - Epoch 50/100, Loss: 0.6351
Client 3 - Epoch 75/100, Loss: 0.5687
Client 3 - Epoch 100/100, Loss: 0.4592

--- Neural Network Weights ---
stack.0.weight: [[ 7.04397261e-02 -3.42991978e-01  1.35358408e-01 -1.29970789e-01
   3.21433634e-01 -1.48006618e-01 -3.70647550e-01 -5.42498417e-02
   6.13159724e-02  3.27399783e-02  4.55933325e-02  1.29824683e-01]
 [ 1.99739859e-01  6.27903640e-02  2.87511736e-01  1.83152363e-01
  -3.08130622e-01  3.38119537e-01  2.61126757e-02 -1.44773602e-01
   2.67507583e-01  1.15865618e-01 -6.81382418e-02  2.47229021e-02]
 [ 1.50400177e-01 -1.52793512e-01 -3.14887524e-01  6.1160933